# 3D GAUSSIAN SPLATTING

## Introduction
 3D Gaussian Splatting is a recent technique for novel view synthesis that achieves real-time rendering with state-of-the-art quality​. It replaces expensive neural networks with a fast point-based representation: the scene is represented as millions of tiny 3D Gaussians (“splats”), each with a position, anisotropic covariance (shape), color (with view-dependent effects), and opacity.

The method has three key components​: 
1. starting from a sparse set of points (e.g. from COLMAP Structure-from-Motion), initialize a radiance field as 3D Gaussians instead of voxels or meshes; 
2. interleave optimization of Gaussian parameters (position, covariance, color) with adaptive density control (adding or removing Gaussians) to progressively refine scene geometry; 
3. use a fast differentiable rasterizer that projects these 3D Gaussians to the image plane and blends them with proper ordering (front-to-back alpha compositing) for real-time rendering​. 


We will reimplement the core of this pipeline step by step in Python/PyTorch, explaining each part in detail and linking to the corresponding parts of the paper.

## Data Processing (Input Preparation)

The first step is to prepare the input data: a set of calibrated images of the scene and an initial sparse 3D point cloud (if available). The sparse points typically come from a Structure-from-Motion (SfM) reconstruction (e.g. COLMAP) of the scene. These SfM points will serve as initial Gaussian locations, providing a rough geometry prior for optimization​. If an initial point cloud is not available (e.g. for synthetic scenes), we can start with a random distribution of points, but using real SfM points speeds up convergence and improves quality.